In [1]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")
%pwd

In [4]:
# raw_data_path = "data ingestion\\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1"
# data_path  = os.path.join(os.getcwd(),raw_data_path)
# data_path

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from heartDiseaseClassification.constant import *
from heartDiseaseClassification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import urllib.request as request
from heartDiseaseClassification import logger
from heartDiseaseClassification.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_progress_hook(self, count, blockSize, totalSize):
        old_percent = 0
        percent = int(count * blockSize * 100 / totalSize)
        if percent > old_percent:
            old_percent = percent
            os.system('cls')
            print(percent, '%')
        if percent == 100:
            os.system('cls')
            print('done!')
            
    def download_file(self):
        '''
        Fetch data from the url
        '''
        logger.info(f"downloading the data from url {self.config.source_URL}")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file,
                reporthook=self.download_progress_hook,

            )
            logger.info(f"{filename}, download! with the following info {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        logger.info(f"extracting the data from {self.config.local_data_file}")
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extraction of the data complete!! at location {unzip_path}")
        

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    logger.debug(f"ingestion_config complete")
    data_ingestion = DataIngestion(config=data_ingestion_config)
    logger.debug(f"data_ingestion create, before dowload file")
    # data_ingestion.download_file()
    logger.debug(f"download file completed")
    data_ingestion.extract_zip_file()
    logger.debug(f"zip file extracted")
except Exception as e:
    raise e

[2024-02-14 11:49:57: INFO: common: yaml files: config.yaml loadded successfully!!]
[2024-02-14 11:49:57: INFO: common: yaml files: params.yaml loadded successfully!!]
[2024-02-14 11:49:57: INFO: common: created director at : artifacts]
[2024-02-14 11:49:57: INFO: common: created director at : artifacts/data_ingestion]
[2024-02-14 11:49:57: INFO: 625064437: extracting the data from artifacts/data_ingestion/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3.zip]


[2024-02-14 11:51:56: INFO: 625064437: Extraction of the data complete!! at location artifacts/data_ingestion]


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e